# <div align="center"><b> ПРОЕКТ №6. РЕКОМЕНДАЦИЯ ТАРИФОВ </b></div> 

**Цель**: проект выполняется для оператора мобильной связи "Мегалайн". Целью проекта является построение модели, предлагающей абонентам один из актуальных тарифов: "Смарт" или "Ультра". Предложение должно формироваться автоматически после анализа моделью данных о пользовательской активности (продолжительности звонков, количества отправленных sms-сообщений, объема израсходованного интернет-траффика). Модель должна обеспечивать не менее 75% правильных ответов.

**Задачи**: 
- исследовать качество различных моделей в результате изменения гиперпараметров;
- проверить модели на вменяемость.

# 1. Подготовительный этап

Согласно заданию на проект, предобработка данных не требуется. В рамках подготовительного этапа будет выполнено следующее:
- импортированы библиотеки;
- сформирован датасет с данными о поведении пользователей;
- исходные данные разделены на обучающую, валидационную и тестовую выборки.

## 1.1 Импорт библиотек и исходных данных

In [1]:
#импортируем библиотеки
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
#снимем ограничение на количество столбцов датафрейма, выводимых на экран
pd.options.display.max_columns = None

In [3]:
#прочитаем файл с исходными данными, сохраним в переменную df
try:
    # local machine
    df = pd.read_csv(r'C:\Users\vizum\Desktop\yandex_practicum\project_6\dataset\users_behavior.csv')
except:
    #yandex practicum server
#     код ревьюера
    df = pd.read_csv('/datasets/users_behavior.csv')
#     df_users = pd.read_csv('/datasets/users_behavior.csv')
    
#выведем на экран первые 5 строк датафрейма
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
#получим общую информацию о df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## 1.2 Разбиение исходных данных на обучающую, валидационную и тестовую выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. Выполним это в два этапа:
- выделим обучающую выборку и совместную выборку для валидации и тестирования;
- разделим совместную выборку для валидации и тестирования на валидационную и тестовую.

После завершения разбивки для каждой выборки выделим набор признаков, а также целевой признак.

Перед разделением выборки определим, сколько пользователей приходится на каждый тариф в исходных данных:

In [5]:
df.groupby(by='is_ultra')['calls'].count().reset_index()

,is_ultra,calls
0,0,2229
1,1,985


Для тарифа Ультра пользователей в 2.26 раз меньше, чем для тарифа Смарт.

**Разделим исходные данные на обучающую и совместную выборку для валидации и тестирования:**

In [6]:
df_train, df_valid_and_test = train_test_split(df, test_size=0.4, random_state=12345, stratify=df['is_ultra'])

In [7]:
#определим соотношение классов в выборке df_train
print('Сooтношение классов в выборке df_train (Смарт/Ультра):', round((df_train['is_ultra'].count()/df_train['is_ultra'].sum()-1), 2))
#определим соотношение классов в выборке df_valid_and_test
print('Сooтношение классов в выборке df_valid_and_test (Смарт/Ультра):', round((df_valid_and_test['is_ultra'].count()/df_valid_and_test['is_ultra'].sum()-1), 2))

Сooтношение классов в выборке df_train (Смарт/Ультра): 2.26
Сooтношение классов в выборке df_valid_and_test (Смарт/Ультра): 2.26


**Разделим совместную выборку для валидации и тестирования на валидационную и тестовую:** <a id='SU'></a>

In [8]:
df_valid, df_test = train_test_split(df_valid_and_test, test_size=0.5, random_state=12345, stratify=df_valid_and_test['is_ultra'])

In [9]:
#определим соотношение классов в выборке df_valid
print('Сooтношение классов в выборке df_valid (Смарт/Ультра):', round((df_valid['is_ultra'].count()/df_valid['is_ultra'].sum()-1), 2))
#определим соотношение классов в выборке df_valid_and_test
print('Сooтношение классов в выборке df_test (Смарт/Ультра):', round((df_test['is_ultra'].count()/df_test['is_ultra'].sum()-1), 2))

Сooтношение классов в выборке df_valid (Смарт/Ультра): 2.26
Сooтношение классов в выборке df_test (Смарт/Ультра): 2.26


In [10]:
df_test['is_ultra'].value_counts()

0    446
1    197
Name: is_ultra, dtype: int64

Соотношение классов в обучающей, валидационной и тестовой выборках такое же, как и в исходной выборке.

**Выделим признаки и целевой признак:**

In [11]:
#разделим обучающую выборку на features и target
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

#проверим результат
display(df_train.head())
display(features_train.head())
target_train.head()

,calls,minutes,messages,mb_used,is_ultra
2294,76.0,484.49,11.0,22454.35,0
1759,51.0,328.88,24.0,20511.93,0
529,105.0,705.05,18.0,22130.74,1
1599,19.0,135.52,48.0,13152.50,0
1868,28.0,159.55,0.0,14780.00,0


,calls,minutes,messages,mb_used
2294,76.0,484.49,11.0,22454.35
1759,51.0,328.88,24.0,20511.93
529,105.0,705.05,18.0,22130.74
1599,19.0,135.52,48.0,13152.50
1868,28.0,159.55,0.0,14780.00


2294    0
1759    0
529     1
1599    0
1868    0
Name: is_ultra, dtype: int64

In [12]:
#разделим валидационную выборку на features и target
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

#проверим результат
display(df_valid.head())
display(features_valid.head())
target_valid.head()

,calls,minutes,messages,mb_used,is_ultra
78,73.0,562.27,11.0,17589.08,0
2024,27.0,147.66,39.0,7545.04,0
2907,164.0,1132.94,16.0,5998.50,1
680,126.0,848.92,57.0,19967.05,0
3078,13.0,60.13,22.0,29289.67,1


,calls,minutes,messages,mb_used
78,73.0,562.27,11.0,17589.08
2024,27.0,147.66,39.0,7545.04
2907,164.0,1132.94,16.0,5998.50
680,126.0,848.92,57.0,19967.05
3078,13.0,60.13,22.0,29289.67


78      0
2024    0
2907    1
680     0
3078    1
Name: is_ultra, dtype: int64

In [13]:
#разделим тестовую выборку на features и target
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

#проверим результат
display(df_test.head())
display(features_test.head())
target_test.head()

,calls,minutes,messages,mb_used,is_ultra
3113,66.0,460.33,0.0,15402.88,0
1459,38.0,312.02,66.0,21645.98,0
2791,60.0,445.03,117.0,10231.26,1
3141,6.0,32.49,12.0,1444.15,1
2380,63.0,472.74,40.0,23709.86,0


,calls,minutes,messages,mb_used
3113,66.0,460.33,0.0,15402.88
1459,38.0,312.02,66.0,21645.98
2791,60.0,445.03,117.0,10231.26
3141,6.0,32.49,12.0,1444.15
2380,63.0,472.74,40.0,23709.86


3113    0
1459    0
2791    1
3141    1
2380    0
Name: is_ultra, dtype: int64

Разделение выборок на признаки и целевой признак выполнено корректно.

# 2. Создание модели рекомендации тарифов

В текущем разделе создадим модель рекомендации тарифов пользователям компании "Мегалайн" с использованием каждого из трех алгоритмов:
- дерева решений;
- случайного леса;
- логистической регрессии.

Для каждого из алгоритмов осуществим поиск наилучшей модели путем изменения гиперпараметров.

## 2.1 Создание модели на основе дерева решений. Исследование качества модели при различных гиперпараметрах

Построим модель рекомендации тарифа на основе дерева решений.
Рассмотрим результат работы модели:
- при различных критериях разделения (gini, entropy);
- при различной глубине дерева (от 1 до 10 включительно);
- при различных минимальных объемах выборки для разделения узла (от 2 до 10 включительно)
- при различных сочетаниях критерия, глубины дерева и минимального объемах выборки для разделения узла.

### 2.1.1 Построение модели на основе дерева решений со значениями по умолчанию <a id='2.1.1'></a>

In [14]:
#создадим и обучим модель
default_dt_model = DecisionTreeClassifier(random_state=12345)
default_dt_model.fit(features_train, target_train)

#сделаем прогноз целевого признака по валидационной выборке и рассчитаем его точность
predictions = default_dt_model.predict(features_valid)
result = accuracy_score(target_valid, predictions) 
print('Доля правильных прогнозов:', result)

Доля правильных прогнозов: 0.7013996889580093


Точность модели со значениями по умолчанию не удовлетворяет требованиям задания.

### 2.1.2 Поиск оптимального критерия разделения

In [15]:
#создадим для перебора список с названиями критериев разделения
criterions = ['gini', 'entropy']

#зададим начальные значения искомых параметров модели
best_model_crit = None
best_result_crit = 0
best_criterion = None

#выполним поиск наилучшего критерия
for criterion in criterions:
    model = DecisionTreeClassifier(criterion=criterion, random_state=12345)
    model.fit(features_train, target_train)
    
    #выполним предсказание и рассчитаем его точность
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions) 
    
    #выведем точность текущей модели
    print(criterion, result)
    
    #сравним точность текущей модели с предыдущим значением. При успехе примем модель за наилучшую
    if result>best_result_crit:
        best_result_crit = result
        best_criterion = criterion
        best_model_crit = model
    
#выведем результат на экран    
print('\nAccuracy лучшей модели:', best_result_crit)
print('Наилучший критерий:', best_criterion)

gini 0.7013996889580093
entropy 0.7387247278382582

Accuracy лучшей модели: 0.7387247278382582
Наилучший критерий: entropy


Точность модели составляет 0.74, что не удовлетворяет требованиям задания.

### 2.1.3 Поиск наилучшей максимальной глубины дерева решений

In [16]:
#зададим начальные значения искомых параметров модели
best_model_depth = None
best_result_depth = 0
best_depth = 0

#выполним поиск наилучшей глубины
for depth in range(1, 11):
    
    #создадим и обучим модель
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train)
    
    #выполним предсказание и рассчитаем его точность
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    
    #сравним точность текущей модели с предыдущим значением. При успехе примем модель за наилучшую
    if result > best_result_depth:
        best_model_depth = model
        best_result_depth = result
        best_depth = depth
        
    #выведем точность текущей модели
    print('глубина =', depth, '; точность =', result)

#выведем результат на экран 
print("\nМаксимальная глубина дерева:", best_depth)
print("Accuracy лучшей модели:", best_result_depth)

глубина = 1 ; точность = 0.7402799377916018
глубина = 2 ; точность = 0.7729393468118196
глубина = 3 ; точность = 0.7776049766718507
глубина = 4 ; точность = 0.7542768273716952
глубина = 5 ; точность = 0.7853810264385692
глубина = 6 ; точность = 0.7744945567651633
глубина = 7 ; точность = 0.7869362363919129
глубина = 8 ; точность = 0.80248833592535
глубина = 9 ; точность = 0.7822706065318819
глубина = 10 ; точность = 0.7729393468118196

Максимальная глубина дерева: 8
Accuracy лучшей модели: 0.80248833592535


При глубине дерева 8 и критерии разделения Джини (установлен по умолчанию) точность модели 0.80, что на текущем этапе удовлетворяет требованиям задания. Однако при глубине дерева 8 возможно переобучение. Проверим:

In [17]:
print('Точность модели с глубиной дерева 8 на обучающей выборке:')
accuracy_score(target_train, best_model_depth.predict(features_train))

Точность модели с глубиной дерева 8 на обучающей выборке:


0.8558091286307054

Действительно, модель переобучилась, при этом для всех рассматриваемых значений глубины дерева точность предсказания на валидационной выборке ниже, чем на тренировочной.

### 2.1.3 Поиск наилучшего минимального объема узлов

In [18]:
#зададим начальные значения искомых параметров модели
best_model_knot = None
best_result_knot = 0
best_knot = 0

#выполним поиск наилучшего минимального объема узлов
for knot in range(2, 11):
    model = DecisionTreeClassifier(random_state=12345, min_samples_split=knot) 
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    
    #выведем точность текущей модели
    print('объем выборки =', knot, '; точность =', result)
    
    #сравним точность текущей модели с предыдущим значением. При успехе примем модель за наилучшую
    if result > best_result_knot:
        best_model_knot = model
        best_result_knot = result
        best_knot = knot

#выведем результат на экран 
print("\nAccuracy лучшей модели:", best_result_knot)
print("Минимальный объем выборки для разделения узла:", best_knot)

объем выборки = 2 ; точность = 0.7013996889580093
объем выборки = 3 ; точность = 0.7169517884914464
объем выборки = 4 ; точность = 0.7216174183514774
объем выборки = 5 ; точность = 0.7262830482115086
объем выборки = 6 ; точность = 0.7247278382581649
объем выборки = 7 ; точность = 0.7231726283048211
объем выборки = 8 ; точность = 0.7278382581648523
объем выборки = 9 ; точность = 0.7262830482115086
объем выборки = 10 ; точность = 0.7262830482115086

Accuracy лучшей модели: 0.7278382581648523
Минимальный объем выборки для разделения узла: 8


Точность модели при изменении минимального объема выборки в узлах меняется незначительно. Максимальная достигнутая точность модели составляет 0.73, что не удовлетворяет требованиям задания.

### 2.1.4 Поиск оптимального сочетания критерия разделения, глубины дерева и объема выборки для разделения

In [19]:
#зададим начальные значения искомых параметров модели
best_model = None
best_result = 0
best_criterion = None
best_depth = 0
best_knot = 0

#поиск наилучшего оптимального соотношения гиперпараметров
for criterion in criterions:
    for depth in range(1, 10):
        for knot in range(2, 11):
            
            #создадим и обучим модель
            model = DecisionTreeClassifier(criterion=criterion, random_state=12345, max_depth=depth, min_samples_split=knot)
            model.fit(features_train, target_train)
            
            #сделаем предсказание и оценим его точность
            predictions = model.predict(features_valid)
            result = accuracy_score(target_valid, predictions) 
    
            #определим гиперпараметры, соответствующие наилучшему результату
            if result > best_result:
                best_model = model
                best_result = result
                best_criterion = criterion
                best_depth = depth
                best_knot = knot

#вывод на экран оптимальных гиперпараметров
print("Accuracy лучшей модели:", best_result)
print('Наилучший критерий:', best_criterion)
print("Максимальная глубина дерева:", best_depth)
print("Минимальный объем выборки для разделения узла:", best_knot)

Accuracy лучшей модели: 0.80248833592535
Наилучший критерий: gini
Максимальная глубина дерева: 8
Минимальный объем выборки для разделения узла: 2


На текущем этапе модель удовлетворяет требованиям задания и полностью соответствует модели, полученной в п. 2.1.3, так как полученные значения наилучшего критерия и минимального объема выборки для разделения узла совпадают со значениями по умолчанию.

**Вывод:**

Модель, для которой не осуществлялась настройка ни одного гиперпараметра, обладает точностью ниже требуемой.
Наилучшей моделью с использованием дерева решений является модель с критерием разделения Джини, максимальной глубине дерева 8, минимальным объемом выборки для разделения узла - 2. Однако, скорее всего, такая модель переобучена.

## 2.2 Создание модели на основе случайного леса. Исследование качества модели при различных гиперпараметрах

Построим модель рекомендации тарифа на основе случайного леса.
Рассмотрим результат работы модели:

- при различном количестве деревьев (от 10 до 50 включительно с шагом 10)
- при различной глубине дерева (от 1 до 10 включительно);

### 2.2.1 Построение модели на основе алгоритма случайного леса со значениями по умолчанию

In [20]:
#создадим и обучим модель
default_rf_model = DecisionTreeClassifier(random_state=12345)
default_rf_model.fit(features_train, target_train)

#сделаем прогноз целевого признака по валидационной выборке и рассчитаем его точность
predictions = default_rf_model.predict(features_valid)
result = accuracy_score(target_valid, predictions) 
print('Доля правильных ответов:', result)

Доля правильных ответов: 0.7013996889580093


Точность модели со значениями по умолчанию ниже заявленной и соответствует модели на основе дерева решений со значениями по умолчанию ([п. 2.1.1](#2.1.1).

### 2.2.2 Поиск наилучшего количества деревьев

In [21]:
#зададим начальные значения искомых параметров модели
best_model_trees = None
best_result_trees = 0
best_trees = 0

#определим наилучшее количество деревьев
for n_trees in range(10, 51, 10):
    
    #создадим и обучим модель
    model = RandomForestClassifier(random_state=12345, n_estimators=n_trees) 
    model.fit(features_train, target_train)
    
    #сделаем предсказание и оценим его точность
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    
    #выведем точность текущей модели
    print('деревьев =', n_trees, '; точность =', result)
    
    #сравним точность текущей модели с предыдущим значением. При успехе примем модель за наилучшую
    if result > best_result_trees:
        best_model_trees = model
        best_result_trees = result
        best_trees = n_trees

#выведем результат на экран 
print('\nAccuracy лучшей модели:', best_result_trees)
print('Наилучшее количество деревьев', best_trees)

деревьев = 10 ; точность = 0.7869362363919129
деревьев = 20 ; точность = 0.7916018662519441
деревьев = 30 ; точность = 0.7931570762052877
деревьев = 40 ; точность = 0.7931570762052877
деревьев = 50 ; точность = 0.8055987558320373

Accuracy лучшей модели: 0.8055987558320373
Наилучшее количество деревьев 50


На текущем этапе полученная модель удовлетворяет требованиям задания (при любом рассмотренном количестве деревьев).

### 2.2.3 Поиск наилучшей максимальной глубины деревьев

In [22]:
#зададим начальные значения искомых параметров модели
best_model_forest_depth = None
best_result_forest_depth = 0
best_forest_dept = 0

#выполним поиск наилучшей максимальной глубины
for depth in range(1, 11):
    
    #создадим и обучим модель
    model = RandomForestClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train)
    
    #сделаем предсказание и оценим его точность    
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)

    #выведем точность текущей модели
    print('глубина =', depth, '; точность =', result)    
    
    #сравним точность текущей модели с предыдущим значением. При успехе примем модель за наилучшую
    if result > best_result_forest_depth:
        best_model_forest_depth = model
        best_result_forest_dept = result
        best_forest_dept = depth

#выведем результат на экран 
print('\nAccuracy лучшей модели:', best_result_forest_dept)
print('Наилучшая максимальная глубина дерева:', best_forest_dept)

глубина = 1 ; точность = 0.7325038880248833
глубина = 2 ; точность = 0.7869362363919129
глубина = 3 ; точность = 0.7962674961119751
глубина = 4 ; точность = 0.7947122861586314
глубина = 5 ; точность = 0.7993779160186625
глубина = 6 ; точность = 0.8040435458786936
глубина = 7 ; точность = 0.8133748055987559
глубина = 8 ; точность = 0.8149300155520995
глубина = 9 ; точность = 0.8133748055987559
глубина = 10 ; точность = 0.8149300155520995

Accuracy лучшей модели: 0.8149300155520995
Наилучшая максимальная глубина дерева: 10


На текущем этапе полученная модель удовлетворяет требованиям задания, однако тоже может быть переучена.

### 2.2.4 Поиск наилучшего сочетания количества деревьев и их максимальной глубины

In [23]:
#зададим начальные значения искомых параметров модели
best_model_trees_and_depth = None
best_result_trees_and_depth = 0
best_trees = 0
best_depth = 0

#выполним поиск оптимальных параметров
for n_trees in range(10, 51, 10):
    for depth in range(1, 11):
        
        #создадим и обучим модель
        model = RandomForestClassifier(random_state=12345, n_estimators=n_trees, max_depth=depth) 
        model.fit(features_train, target_train)
        
        #сделаем предсказание и оценим его точность
        predictions = model.predict(features_valid) 
        result = accuracy_score(target_valid, predictions)
    
        #сравним точность текущей модели с предыдущим значением. При успехе примем модель за наилучшую
        if result > best_result_trees_and_depth:
            best_model_trees_and_depth = model
            best_result_trees_and_depth = result
            best_trees = n_trees
            best_depth = depth

#выведем результат на экран 
print('Accuracy лучшей модели:', best_result_trees)
print('Наилучшее количество деревьев', best_trees)
print('Наилучшая максимальная глубина дерева:', best_forest_dept)

Accuracy лучшей модели: 0.8055987558320373
Наилучшее количество деревьев 40
Наилучшая максимальная глубина дерева: 10


**Вывод:**

Все модели, кроме модели со всеми гиперпараметрами по умолчанию, удовлетворяют требованию задания. Наилучшей моделью с использованием случайного леса является модель с максимальной глубиной деревьев 10 и остальными гиперпараметрами, заданными по умолчанию. При этом наилучшая модель на основе случайного леса превосходит по точности все модели, созданные с использованием дерева решений. Как и в случае с деревом решений, наилучшая модель может быть переучена. 

## 2.3 Создание модели на основе логистической регрессии. Исследование качества модели при различных гиперпараметрах

Построим модель рекомендации тарифа на основе логистической регрессии.
Рассмотрим результат работы модели при различных значениях параметра solver.

### 2.3.1 Построение модели рекомендации тарифов на основе логистической регрессии. Все гиперпараметры заданы значениями по умолчанию

In [24]:
#создадим и обучим модель
default_lr_model = LogisticRegression(random_state=12345)
default_lr_model.fit(features_train, target_train)

#сделаем предсказание и оценим его точность
predictions = default_lr_model.predict(features_valid)
result = accuracy_score(target_valid, predictions) 
print('Доля правильных ответов:', result)

Доля правильных ответов: 0.7387247278382582


Точность "дефолтной" модели на основе логистической регрессии ниже требуемой.

### 2.3.2 Поиск наилучшего алгоритма

In [25]:
#создадим для перебора список с названиями алгоритмов
algorythms = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

#зададим начальные значения искомых параметров модели
best_model_alg = None
best_result_alg = 0
best_alg = None

#выполним поиск наилучшего алгоритма
for algorythm in algorythms:
    
    #создадим и обучим модель
    model = LogisticRegression(solver=algorythm, random_state=12345)
    model.fit(features_train, target_train)
    
    #сделаем предсказания и оценим их точность
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions) 
    
    #выведем точность текущей модели
    print('алгоритм =', algorythm, '; точность =', result)   
    
    #сравним точность текущей модели с предыдущим значением. При успехе примем модель за наилучшую
    if result>best_result_alg:
        best_result_alg = result
        best_alg = algorythm
        best_model_alg = model
    
#выведем результат на экран    
print("\nAccuracy лучшей модели:", best_result_alg)
print('Наилучший алгоритм:', best_alg)

алгоритм = newton-cg ; точность = 0.7387247278382582
алгоритм = lbfgs ; точность = 0.7387247278382582
алгоритм = liblinear ; точность = 0.71850699844479
алгоритм = sag ; точность = 0.6936236391912908
алгоритм = saga ; точность = 0.6936236391912908

Accuracy лучшей модели: 0.7387247278382582
Наилучший алгоритм: newton-cg


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Наилучшей точностью для решения задачи рекомендации тарифов на основе пользовательской активности с помощью логистической регрессии обладают алгоритмы *newton-cg* и *lbfgs*, показывающие одинаковую точность, наихудшей - алгоритмы *sag* и *saga* (также равноточные в рамках текущей задачи).

**Вывод:**

Ни одна из рассмотренных моделей на основе логистической регрессии не удовлетворяет требованиям точности.

**ОБЩИЙ ВЫВОД ПО РАЗДЕЛУ 2:**
Наилучшую точность на валидационной выборке (81% правильных ответов) показала модель best_result_forest_depth, созданная на основе случайного леса с максимальной глубиной дерева 10.Далее проверим качество этой модели на тестовой выборке.

# 3. Проверка качества модели на тестовой выборке

В текущем разделе проверим на тестовой выборке качество наилучшей модели best_model_forest_depth, полученной на основе случайного леса  с максимальной глубиной дерева 10. Остальные параметры заданы по умолчанию.

In [26]:
#сделаем прогноз
predictions = best_model_forest_depth.predict(features_test)

#рассчитаем долю верных ответов
result = accuracy_score(target_test, predictions)
print('Доля правильных ответов:', result)

Доля правильных ответов: 0.8118195956454122


**Вывод:** точность лучшей модели на основе случайного леса на тестовых данных практически не отличается от проверки на валидационной выборке и удовлетворяет требованиям задания.

# 4. Проверка моделей на вменяемость

Для оценки адекватности полученных моделей сравним их точность с наилучшей константной моделью. Так как для обучающей, валидационной и тестовой выборок объектов с тарифом Смарт в 2.26 раз больше, чем объектов с тарифом Ультра ([раздел 1.2](#SU)), то предсказание наилучшей константной модели будет состоять из нулей. Рассчитаем долю правильных ответов для такой модели на тестовой выборке:

In [27]:
print('Точность константной модели:', 1-target_test.mean())

Точность константной модели: 0.6936236391912909


Точность наилучших моделей, реализованных с помощью дерева решений и с помощью случайного леса превышает точность наилучшей константной модели в 1.13 и и в 1.17 раз соответственно (на тестовых данных). Точность лучшей модели, реализованной с помощью логистической регрессии, превышает точность случайной модели (на тестовых данных) всего в 1.08 раз.

# 5. Общий вывод

В качестве итоговой модели для рекомендации тарифов можно принять модель *best_model_forest_depth*, реализованную с помощью метода случайного леса и обеспечивающую 81% правильных ответов на тестовых данных. Полученная точность удовлетворяет требуемой точности модели (не менее 75% верных ответов) и превышает точность наилучшей константной модели, что говорит о вменяемости модели.